In [ ]:
import sys
# 'sys.path' is a list of absolute path strings
sys.path.append('/home/ricky/rkw0k/PyPortfolioOpt')

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt.risk_models import sample_cov
from pypfopt.hierarchical_portfolio import HRPOpt
from pypfopt import plotting

def get_historical_prices(tickers):
    """
    Fetches historical adjusted close price data for the given tickers using yfinance.
    Data is fetched from 2020-01-01 to the current date.
    """
    print("Fetching live data from yfinance (Start Date: 2020-01-01)...")
    
    # Download data from 2020-01-01 to today (default end date)
    df = yf.download(tickers, start="1970-01-01", progress=False)
    df_prices = df['Close']
    # Drop rows where any ticker has a missing price (e.g., non-trading days)
    df_prices = df_prices.dropna()
    print(f"--- Historical Price Data Fetched ---")
    print(f"Start Date: {df_prices.index.min().strftime('%Y-%m-%d')}, End Date: {df_prices.index.max().strftime('%Y-%m-%d')}")
    print(f"Total trading days: {len(df_prices)}")
    return df_prices

def calculate_risk_parity_weights(tickers):
    """
    Calculates the Hierarchical Risk Parity (HRP) allocation weights for a list of tickers.
    HRP is a sophisticated, robust alternative to traditional Risk Parity (Equal Risk Contribution).

    :param tickers: A list of ticker symbols (str).
    :return: A tuple of (weights_dict, volatility_float).
    """
    # 1. Get Historical Price Data
    df_prices = get_historical_prices(tickers)
    # 2. Calculate Returns
    df_returns = df_prices.pct_change().dropna()
    # 3. Perform Hierarchical Risk Parity Optimization
    try:
        hrp = HRPOpt(df_returns)
        
        # Calculate the weights
        hrp_weights = hrp.optimize()
        
        # Clean weights for display (removes tiny errors, converts to dict of tickers/weights)
        clean_weights = hrp.clean_weights()

        # Calculate metrics for concise display
        expected_return, volatility, sharpe_ratio = hrp.portfolio_performance(verbose=False)
        # Optional: Plot the dendrogram for visual inspection of clusters
        plotting.plot_dendrogram(hrp, save_fig=False, show_fig=False) 
        return clean_weights, volatility

    except Exception as e:
        print(f"An error occurred during optimization: {e}")
        return {}, None


# The five ticker symbols for which to calculate allocation
TICKERS = ["TLT", "VIOV", "VTI", "GLD", "SHY"]

# Calculate weights and volatility
weights_dict, volatility = calculate_risk_parity_weights(TICKERS)

if weights_dict:
    # Convert dictionary to Pandas Series for nice formatting
    weights_series = pd.Series(weights_dict)
    
    print("\n--- Risk Parity Allocation (HRP) ---")
    print(f"TICKERS: {TICKERS}")
    
    if volatility is not None:
        print(f"Annual Volatility: {volatility:.2%}")
    
    print(f"Total Weight: {weights_series.sum():.4f}\n")
    
    print("Final Portfolio Weights (Tickers and Weights):")
    # Use to_string() for clean, formatted output that works well in both console and Jupyter
    print(weights_series.to_string(float_format="{:.4f}".format))
    df_prices = get_historical_prices(TICKERS)
else:
    print("Could not calculate weights.")
    

In [ ]:
# df_prices.head(50)

In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import expected_returns, risk_models
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import plotting

def get_historical_prices(tickers):
    """
    Fetches historical adjusted close price data for the given tickers using yfinance.
    Data is fetched from 2020-01-01 to the current date.
    """

    # Download data from 2020-01-01 to today (default end date)
    df = yf.download(tickers, start="1970-01-01", progress=False)
    df_prices = df['Close']
    df_prices = df_prices.dropna()
    
    print(f"Start Date: {df_prices.index.min().strftime('%Y-%m-%d')}, End Date: {df_prices.index.max().strftime('%Y-%m-%d')}")
    print(f"Total trading days: {len(df_prices)}")
    
    return df_prices

def calculate_min_volatility_weights(tickers):
    """
    Calculates the Minimum Volatility portfolio weights for a list of tickers.
    This method uses the EfficientFrontier solver to find the portfolio with the lowest risk.

    :param tickers: A list of ticker symbols (str).
    :return: A tuple of (weights_dict, volatility_float).
    """
    # 1. Get Historical Price Data
    df_prices = get_historical_prices(tickers)

    # 2. Calculate Expected Returns and Sample Covariance Matrix
    
    # Mean Historical Return is used for the Min Vol calculation (though it only needs the covariance)
    mu = expected_returns.mean_historical_return(df_prices)
    # The standard sample covariance matrix is used for risk estimation
    S = risk_models.sample_cov(df_prices)

    # 3. Perform Minimum Volatility Optimization
    
    try:
        # Initialize EfficientFrontier with expected returns and covariance
        ef = EfficientFrontier(mu, S)
        
        # Calculate the weights for the minimum volatility portfolio
        # We enforce no short-selling (weights >= 0) by default
        ef.min_volatility()
        
        # Clean weights for display (removes tiny errors, converts to dict of tickers/weights)
        clean_weights = ef.clean_weights()

        # Calculate metrics for concise display
        # The method portfolio_performance() returns (expected_return, volatility, sharpe_ratio)
        expected_return, volatility, sharpe_ratio = ef.portfolio_performance(verbose=False)
        
        return clean_weights, volatility

    except Exception as e:
        print(f"An error occurred during optimization: {e}")
        return {}, None

# --- EXECUTION ---
if __name__ == "__main__":
    # The five ticker symbols for which to calculate allocation
    TICKERS = ["TLT", "VTI", "GLD"]
    
    # Calculate weights and volatility using the Min Vol method
    weights_dict, volatility = calculate_min_volatility_weights(TICKERS)
    
    if weights_dict:
        # Convert dictionary to Pandas Series for nice formatting
        weights_series = pd.Series(weights_dict)
        
        print("\n--- Minimum Volatility Allocation ---")
        print(f"TICKERS: {TICKERS}")
        
        if volatility is not None:
            print(f"Annual Volatility: {volatility:.2%}")
        
        print(f"Total Weight: {weights_series.sum():.4f}\n")
        
        print("Final Portfolio Weights (Tickers and Weights):")
        # Use to_string() for clean, formatted output that works well in both console and Jupyter
        print(weights_series.to_string(float_format="{:.4f}".format))
    
    else:
        print("Could not calculate weights.")

Fetching live data from yfinance (Start Date: 2020-01-01)...


/tmp/ipykernel_18673/2928662419.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start="1970-01-01", progress=False)


--- Historical Price Data Fetched ---
Start Date: 2004-11-18, End Date: 2025-11-12
Total trading days: 5280

--- Minimum Volatility Allocation ---
TICKERS: ['TLT', 'VTI', 'GLD']
Annual Volatility: 9.17%
Total Weight: 1.0000

Final Portfolio Weights (Tickers and Weights):
GLD   0.1826
TLT   0.4869
VTI   0.3305


In [13]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import risk_models
from pypfopt.hierarchical_portfolio import HRPOpt 
from pypfopt import plotting

def get_historical_prices(tickers):
    """
    Fetches historical adjusted close price data for the given tickers using yfinance.
    Data is fetched from 1970-01-01 (as specified by the user's test).
    """
    print("Fetching live data from yfinance (Start Date: 1970-01-01)...")
    
    # Download data from 1970-01-01 to today (default end date)
    df = yf.download(tickers, start="2015-01-01", progress=False)
    
    df_prices = df['Close']
    
    # Drop rows where any ticker has a missing price (e.g., non-trading days)
    df_prices = df_prices.dropna()
    
    print(f"--- Historical Price Data Fetched ---")
    print(f"Start Date: {df_prices.index.min().strftime('%Y-%m-%d')}, End Date: {df_prices.index.max().strftime('%Y-%m-%d')}")
    print(f"Total trading days: {len(df_prices)}")
    
    return df_prices

def calculate_hrp_weights(tickers):
    """
    Calculates the Hierarchical Risk Parity (HRP) portfolio weights.
    HRP is a stable, cluster-based method that achieves risk parity without external solvers.

    :param tickers: A list of ticker symbols (str).
    :return: A tuple of (weights_dict, volatility_float).
    """
    # 1. Get Historical Price Data
    df_prices = get_historical_prices(tickers)
    # 2. Calculate Returns and Covariance Matrix
    # HRP needs the covariance matrix for risk calculation
    S = risk_models.sample_cov(df_prices) # Standard sample covariance matrix
    # 3. Perform Hierarchical Risk Parity (HRP) Optimization
    
    try:
        # Initialize HRPOpt with the covariance matrix
        hrp = HRPOpt(df_prices.pct_change().dropna()) # HRPOpt takes returns directly
        
        # Calculate the HRP weights
        hrp.optimize() # Correct method is optimize()
        
        # Clean weights for display (removes tiny errors, converts to dict of tickers/weights)
        clean_weights = hrp.clean_weights()

        # The method portfolio_performance() returns (expected_return, volatility, sharpe_ratio)
        expected_return, volatility, sharpe_ratio = hrp.portfolio_performance(verbose=False)
        return clean_weights, volatility
    except Exception as e:
        print(f"An error occurred during HRP optimization: {e}")
        return {}, None


# The five ticker symbols for which to calculate allocation
TICKERS = ["TLT", "VTI", "GLD"]

# Calculate weights and volatility using the HRP method
weights_dict, volatility = calculate_hrp_weights(TICKERS)

if weights_dict:
    # Convert dictionary to Pandas Series for nice formatting
    weights_series = pd.Series(weights_dict)
    
    print("\n--- Hierarchical Risk Parity (HRP) Allocation ---")
    print(f"TICKERS: {TICKERS}")
    
    if volatility is not None:
        print(f"Annual Volatility: {volatility:.2%}")
    
    print(f"Total Weight: {weights_series.sum():.4f}\n")
    
    print("Final Portfolio Weights (Tickers and Weights):")
    # Use to_string() for clean, formatted output that works well in both console and Jupyter
    print(weights_series.to_string(float_format="{:.4f}".format))

else:
    print("Could not calculate weights.")

Fetching live data from yfinance (Start Date: 1970-01-01)...
--- Historical Price Data Fetched ---
Start Date: 2015-01-02, End Date: 2025-11-12
Total trading days: 2733

--- Hierarchical Risk Parity (HRP) Allocation ---
TICKERS: ['TLT', 'VTI', 'GLD']
Annual Volatility: 9.59%
Total Weight: 1.0000

Final Portfolio Weights (Tickers and Weights):
GLD   0.3587
TLT   0.3399
VTI   0.3014


/tmp/ipykernel_18673/3279812885.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start="2015-01-01", progress=False)
